# Power manifold results

## Setup

In [7]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score


## Load results

In [8]:
show_all = True
n_runs = 3
remove_all_results_with_nans = True

In [9]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label, shortlist=False):
    if show_all or shortlist:
        algo_filenames.append(filename)
        algo_additionals.append(add)
        algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))
    

add_algo("flow", "_march", "EF")
add_algo("pie", "_march", "PIE")
add_algo("mf", "_march", "MLF-S")
add_algo("mf", "_alternate_march", "MLF-A")
add_algo("gamf", "_march", "MLF-OT")
add_algo("gamf", "_alternate_march", "MLF-OTA")
add_algo("emf", "_march", "EMLF-S")
add_algo("emf", "_alternate_march", "EMLF-A")


In [10]:
def load(name, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)
            try:
                this_result = np.load(
                    "{}/{}_2_power{}{}_{}.npy".format(
                        result_dir, algo_filename, algo_add, run_str, name
                    )
                )
                if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                    algo_results.append(this_result.reshape(shape))
                else:
                    algo_results.append(np.nan*np.ones(shape))
                    
            except FileNotFoundError as e:
                print(e)
                if shape is None:
                    algo_results.append(None)
                else:
                    algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        
    return all_results


model_gen_x = load("samples", None, numpyfy=False)
model_gen_logp = load("samples_likelihood", (10000,))
model_gen_distance = load("samples_manifold_distance", (10000,))
model_test_logp = load("model_log_likelihood_test", (101, 1000,))
model_test_reco_error = load("model_reco_error_test", (1000,))
model_ood_logp = load("model_log_likelihood_ood", (101, 1000,))
model_ood_reco_error = load("model_reco_error_ood", (1000,))
model_posterior_samples = load("posterior_samples", (5000, 1,))
model_mmds = load("mmd", (1,))


[Errno 2] No such file or directory: '../data/results/emf_2_power_march_run2_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/emf_2_power_march_run2_mmd.npy'


In [11]:
def remove_nans(*results, label=None):
    passes = all([np.all(np.isfinite(result)) for result in results])
    
    if passes:
        return results
    else:
        if label is not None:
            print(label, "removed:", [np.all(np.isfinite(result)) for result in results])
        return [np.nan * np.ones_like(result) for result in results]


def remove_nans_from_lists(*raws):
    # raws[quantity][algo]
    n_quantities = len(raws)
    n_algos = len(raws[0])
    
    for raw in raws:
        assert len(raw) == n_algos
    
    cleans = [[[] for _ in range(n_algos)] for _ in range(n_quantities)]
    
    for i in range(n_algos):
        for k in range(n_runs):
            clean = remove_nans(*[raw[i][k] for raw in raws], label="{}, run {}".format(algo_labels[i], k))
            for j in range(n_quantities):
                cleans[j][i].append(clean[j])
            
    cleans = [np.array(clean) for clean in cleans]
    
    # cleans[quantity][algo]
    return cleans
   
    
if remove_all_results_with_nans:         
    raw = [model_gen_logp, model_gen_distance, model_test_logp, model_ood_logp, model_ood_reco_error, model_posterior_samples, model_mmds]
    clean = remove_nans_from_lists(*raw)
    model_gen_logp, model_gen_distance, model_test_logp, model_ood_logp, model_ood_reco_error, model_posterior_samples, model_mmds = clean


MLF-S, run 0 removed: [True, True, False, False, True, True, True]
MLF-S, run 1 removed: [True, True, False, False, True, True, True]
MLF-S, run 2 removed: [True, True, False, False, True, True, True]
MLF-A, run 2 removed: [True, True, False, False, True, True, True]
EMLF-S, run 0 removed: [True, True, False, False, True, True, True]
EMLF-S, run 1 removed: [True, True, False, False, True, True, True]
EMLF-S, run 2 removed: [True, True, False, False, True, False, False]
EMLF-A, run 0 removed: [True, True, False, False, True, True, True]
EMLF-A, run 1 removed: [True, True, False, False, True, True, True]
EMLF-A, run 2 removed: [True, True, False, False, True, True, True]


In [6]:
def load_truth(name, samples=True):
    if samples:
        return np.asarray([
            np.load("../data/samples/power/{}{}.npy".format(
                name, run_str
            ))
            for run_str in [""] + ["_run{}".format(i) for i in range(1, n_runs)]
        ])
    else:
        return np.asarray([
            np.load("../data/results/truth_power{}_{}.npy".format(
                run_str, name
            ))
            for run_str in [""] + ["_run{}".format(i) for i in range(1, n_runs)]
        ])

test_x = load_truth("x_test", True)
test_distance = np.zeros((test_x.shape[0], 1))
test_logp = load_truth("true_log_likelihood_test", False)
true_posterior_samples = load_truth("posterior_samples", False)

param_grid = np.linspace(-1, 1, 101)


## Calculate metrics

In [7]:
n_observed = 20
min_logp = -100.
max_distance = 10.

model_gen_mean_logp = np.mean(np.clip(model_gen_logp, min_logp, None), axis=2)
model_gen_mean_distance = np.mean(np.clip(model_gen_distance, None, max_distance), axis=2)
model_observed_nll = -2. * np.sum(model_test_logp[:,:,:,:n_observed], axis=-1)
model_test_mean_reco_error = np.mean(np.clip(model_test_reco_error, None, max_distance), axis=2)

test_mean_logp = np.mean(np.clip(test_logp, min_logp, None), axis=1)
test_mean_distance = np.mean(np.clip(test_distance, None, max_distance), axis=1)
true_observed_nll = -2. * np.sum(test_logp[:,:,:n_observed], axis=-1)


In [8]:
def calculate_roc_auc(x0, x1):
    assert x0.shape == x1.shape
    old_shape = x0.shape[:-1]
    x0 = x0.reshape(-1, x0.shape[-1])
    x1 = x1.reshape(-1, x1.shape[-1])
    
    aucs = []
    for x0_, x1_ in zip(x0, x1):
        if not np.all(np.isfinite(np.hstack((x0_, x1_)))):
            aucs.append(np.nan)
            continue
            
        auc = roc_auc_score(
            np.hstack((np.zeros(x0_.shape[0], dtype=np.int), np.ones(x1_.shape[0], dtype=np.int))),
            np.hstack((x0_, x1_)),
        )
        auc_flipped = roc_auc_score(
            np.hstack((np.zeros(x0_.shape[0], dtype=np.int), np.ones(x1_.shape[0], dtype=np.int))),
            - np.hstack((x0_, x1_)),
        )
        aucs.append(max(auc, auc_flipped))
        
    aucs = np.asarray(aucs)
    aucs = aucs.reshape(old_shape)
    return aucs


model_auc_logp = calculate_roc_auc(model_test_logp[:,:,50,:], model_ood_logp[:,:,50,:])
model_auc_err = calculate_roc_auc(model_test_reco_error, model_ood_reco_error)
model_auc_use_err = (model_auc_err > model_auc_logp)
model_auc = np.maximum(model_auc_err, model_auc_logp)


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


## Best metrics

In [9]:
best_mmd = np.nanargmin(np.nanmean(model_mmds, axis=1))
print(algo_labels[best_mmd])

best_dist = np.nanargmin(np.nanmean(model_gen_mean_distance, axis=1))
print(algo_labels[best_dist])

best_auc = np.nanargmax(np.nanmean(model_auc, axis=1))
print(algo_labels[best_auc])

model_test_mean_reco_error_ = np.copy(model_test_mean_reco_error)
model_test_mean_reco_error_[model_test_mean_reco_error_ < 1.e-3] = 1000.
best_recoerr = np.nanargmin(np.nanmean(model_test_mean_reco_error_, axis=1))
print(algo_labels[best_recoerr])

MLF-A
MLF-A
EF
MLF-A


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()


## Print metrics

In [10]:
def print_results(
    include_err=False, include_n_runs=False, include_runs=False,
    l_label=max([len(l) for l in algo_labels]), l_mean=5, l_err=3,
    median=False, latex=False
):
    # How to format the numbers
    l_result = l_mean + int(include_err) * (2 + l_err) + int(include_n_runs) * 4 + int(include_runs) * (3 + 3*l_mean + 2*2)
    l_total = l_label + 1 + 4 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    def _f(val, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        if median:
            result = "{:>{}.{}f}".format(np.nanmedian(val), l_mean, l_mean - 2)
        else:
            result = "{:>{}.{}f}".format(np.nanmean(val), l_mean, l_mean - 2)
            
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        if include_err:
            result += "({:0>{}d})".format(int(10**l_err * np.nanstd(val)), l_err)
        if include_n_runs:
            result += " [{:1n}]".format(np.sum(np.isfinite(val)))
        if include_runs:
            result += " [{:>{}.{}f}, ".format(float(val[0]), l_mean, l_mean - 2)
            result += "{:>{}.{}f}, ".format(float(val[1]), l_mean, l_mean - 2)
            result += "{:>{}.{}f}]".format(float(val[2]), l_mean, l_mean - 2)
            
        if (not latex) and best:
            result += "*"
        elif (not latex) and (not best):
            result += " "
            
        return result
    
    # Header
    print(
        "{2:<{0}.{0}s} {7} {3:>{1}.{1}s} {7} {4:>{1}.{1}s} {7} {5:>{1}.{1}s} {7} {6:>{1}.{1}s} {8}".format(
            l_label, l_result, "", "Man. dist.", "Reco err.", "Post. MMD", "OOD AUC", col_divider, line_end
        )
    )
    print(block_divider)

    # Iterate over methods
    for i, (label, dist, mmd, auc, recoerr) in enumerate(zip(
        algo_labels, model_gen_mean_distance, model_mmds, model_auc, model_test_mean_reco_error
    )):
        # Divider
        if i in algo_dividers and show_all:
            print(block_divider)
            
        # Print results
        print("{1:<{0}.{0}s} {6} {2}{6} {5}{6} {4}{6} {3} {7}".format(
            l_label, label,
            _f(dist, i==best_dist), _f(auc, i==best_auc),  _f(mmd, i==best_mmd), _f(recoerr, i==best_recoerr),
            col_divider, line_end
        ))


In [11]:
print_results(include_runs=True)

        |                  Man. dist. |                   Reco err. |                   Post. MMD |                     OOD AUC 
--------------------------------------------------------------------------------------------------------------------------------
EF      | 0.006 [0.009, 0.003, 0.005] | 0.000 [0.000, 0.000, 0.000] | 0.177 [0.476, 0.026, 0.028] | 0.991 [0.990, 0.991, 0.991]* 
PIE     | 0.025 [0.006, 0.065, 0.005] | 1.353 [1.699, 0.753, 1.608] | 0.070 [0.121, 0.014, 0.075] | 0.974 [0.985, 0.972, 0.964]  
MLF-S   |                             | 0.085 [0.225, 0.011, 0.018] |                             |                              
MLF-A   | 0.002 [0.002, 0.002,   nan]*| 0.004 [0.002, 0.001, 0.008]*| 0.010 [0.001, 0.020,   nan]*| 0.989 [0.987, 0.992,   nan]  
MLF-OT  | 0.105 [0.086, 0.139, 0.089] | 0.409 [0.649, 0.433, 0.145] | 0.277 [0.553, 0.151, 0.126] | 0.634 [0.607, 0.592, 0.703]  
MLF-OTA | 0.214 [0.413, 0.142, 0.085] | 1.135 [1.952, 1.206, 0.248] | 0.221 [0.515, 0.113, 0

In [12]:
print_results(include_err=True)

        | Man. dist. |  Reco err. |  Post. MMD |    OOD AUC 
------------------------------------------------------------
EF      | 0.006(002) | 0.000(000) | 0.177(211) | 0.991(000)* 
PIE     | 0.025(028) | 1.353(425) | 0.070(043) | 0.974(008)  
MLF-S   |            | 0.085(099) |            |             
MLF-A   | 0.002(000)*| 0.004(003)*| 0.010(009)*| 0.989(002)  
MLF-OT  | 0.105(024) | 0.409(206) | 0.277(195) | 0.634(049)  
MLF-OTA | 0.214(143) | 1.135(697) | 0.221(210) | 0.595(042)  
EMLF-S  |            | 0.307(301) |            |             
EMLF-A  |            | 1.653(424) |            |             


In [13]:
print_results(include_err=True, latex=True)

        & Man. dist. &  Reco err. &  Post. MMD &    OOD AUC \\
\midrule
EF      & 0.006(002)& 0.000(000)& 0.177(211)& \textbf{0.991}(000) \\
PIE     & 0.025(028)& 1.353(425)& 0.070(043)& 0.974(008) \\
MLF-S   & & 0.085(099)& &  \\
MLF-A   & \textbf{0.002}(000)& \textbf{0.004}(003)& \textbf{0.010}(009)& 0.989(002) \\
MLF-OT  & 0.105(024)& 0.409(206)& 0.277(195)& 0.634(049) \\
MLF-OTA & 0.214(143)& 1.135(697)& 0.221(210)& 0.595(042) \\
EMLF-S  & & 0.307(301)& &  \\
EMLF-A  & & 1.653(424)& &  \\
